In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd
import time
import sklearn
import tensorflow as tf

print(tf.__version__)

for module in np, sklearn, tf.keras, pd:
    print(module.__name__, module.__version__)

2.1.0
numpy 1.18.1
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
pandas 1.0.1


In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

In [ ]:
import pprint
pprint.pprint(housing.data[0:5])
pprint.pprint(housing.target[0:5])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scalered = scaler.fit_transform(x_train)
x_valid_scalered = scaler.transform(x_valid)
x_test_scalered = scaler.transform(x_test)
print(x_train_scalered.shape)
print(len(x_train_scalered.shape))
print(type(x_train_scalered))

In [ ]:
# class Net(tf.keras.Model):
#     def __init__(self):
#         super(Net, self).__init__()
#         
#         self.hidden1_layer = tf.keras.layers.Dense(30, activation='relu')
#         self.hidden2_layer = tf.keras.layers.Dense(30, activation='relu')
#         self.output_layer = tf.keras.layers.Dense(1)
#         
#     def call(self, inputs, training=None, mask=None):
#         hid1 = self.hidden1_layer(inputs)
#         hid2 = self.hidden2_layer(hid1)
#         concat = tf.keras.layers.concatenate([inputs, hid2])
#         output = self.output_layer(concat)
#         return output
        
def build_model(hidden_layers = 1, layer_size = 30, learning_rate = 1e-3):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(layer_size, activation='relu', input_shape=x_train.shape[1:]))
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(layer_size, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
sklearn_model = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

# history = sklearn_model.fit(x_train_scalered, y_train, epochs=100, validation_data=(x_valid_scalered, y_valid), callbacks=callbacks)

    
    

#         
# histories = []
# for lr in [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2]:
#     model = Net()
#     optimizer =tf.keras.optimizers.SGD(lr)
#     model.build(input_shape=(None,8))
#     model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
#     callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
#     history = model.fit(x_train_scalered, y_train, validation_data=(x_valid_scalered, y_valid), epochs=100, callbacks=callbacks)
#     histories.append(history)
# print(histories)



# inputs = tf.keras.Input(shape=x_train_scalered.shape[1:])
# hidden1 = tf.keras.layers.Dense(30, activation='relu')(inputs)
# hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
# 
# concat = tf.keras.layers.concatenate([inputs, hidden2])
# output = tf.keras.layers.Dense(1)(concat)
# 
# model = tf.keras.models.Model([inputs], [output])


# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(30, activation='relu', input_shape=x_train_scalered.shape[1:]),
#     tf.keras.layers.Dense(1)
# ])

# model.summary()


# def plot_lerning_curves(history):
#     pd.DataFrame(history.history).plot(figsize=(8, 5))
#     plt.grid(True)
#     plt.gca().set_ylim(0, 1)
#     plt.show()
# 
# plot_lerning_curves(history)

In [ ]:
# model.evaluate(x_test_scalered, y_test, verbose=2)
from scipy.stats import reciprocal

param_distribution = {
    "hidden_layers" : [1, 2, 3, 4],
    "layer_size" : np.arange(1,20),
    "learning_rate" : reciprocal(1e-4, 1e-2),
}

from sklearn.model_selection import RandomizedSearchCV

random_search_cv = RandomizedSearchCV(sklearn_model, param_distribution, n_iter=10, cv=3, n_jobs=1)
random_search_cv.fit(x_train_scalered, y_train, epochs=100, validation_data=(x_valid_scalered, y_valid), callbacks=callbacks)